<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/notebooks/2021_0531ccap_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021年5月版 word2vec デモ

- date: 2021_0510
- filename: 2021_0530ccap_word2vec.pynb
- author: 浅川伸一
- 概要:

In [31]:
import numpy as np
# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

In [ ]:
%time
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
# reference: https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n
!pip install mecab-python3

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [33]:
# 動作確認
import MeCab
neologd_path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = MeCab.Tagger(neologd_path +' -Oyomi')
print(m.parse('鬼滅の刃'))

キメツノヤイバ



In [ ]:
#訓練済 word2vec ファイルの取得
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb" -O 2021_05jawiki_hid128_win10_neg10_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M" -O 2021_05jawiki_hid128_win10_neg10_sgns.bin.gz && rm -rf /tmp/cookies.txt
!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s" -O 2021_05jawiki_hid200_win20_neg20_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7" -O 2021_05jawiki_hid200_win20_neg20_sgns.bin.gz && rm -rf /tmp/cookies.txt

In [ ]:
%time
#直上のセルで取得したファイルの読み込み
# word2vec データ処理のため gensim を使う
import os
import sys
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

print('# word2vec データの読み込み')
print('# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...', end="")
# ファイルの所在に応じて変更してください
w2v_base = '.'
#w2v_file='2021_05jawiki_hid128_win10_neg10_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid128_win10_neg10_sgns.bin.gz'
w2v_file='2021_05jawiki_hid200_win20_neg20_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid200_win20_neg20_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file, 
                                                encoding='utf-8', 
                                                unicode_errors='replace',
                                                binary=True) 

w2v = asakawa_w2v
print('done')

In [ ]:
#以下は操作例：単語の後の数字はコサイン類似度を表している
word = '人工知能' #入力単語
topn = 20        #類似度が高い順に何語出力するか
word = input('単語を入力してください: ')
w2v.most_similar(word,topn=topn)

コサイン類似度とは，ベクトル間の類似度を角度 $\theta$ として表現したもの。

$$
\text{$\mathbf{x}$ と $\mathbf{y}$ とのコサイン類似度 $\theta$}=\frac{(\mathbf{x}\cdot\mathbf{y})}{|\mathbf{x}||\mathbf{y}|}
$$

ただし上式では $-1\le\theta\le1$ であるので，$[0,1]$ の範囲になるように，$\theta$ に 1 を加えて 0.5 倍してある。

In [ ]:
w2v.most_similar(positive=('王','女'), negative=('男'),topn=topn)

In [ ]:
w2v.most_similar(positive=('早稲田大学','関西'), negative=('関東'))

In [ ]:
w2v.most_similar(positive=('ジャイアンツ','アメリカ'), negative=('日本'))

In [ ]:
w2v.most_similar(positive=('鬼滅の刃'))

In [ ]:
!pip install jaconv       #単語をカタカナに変換するために使用するライブラリ
!pip install Levenshtein  #レーベンシュタイン距離を算出するライブラリ
import jaconv
import Levenshtein

In [75]:
# word2vec の類似語上位 topn 語の読みを元にレーベンシュタイン距離を算出して表示
topn=10
target = input('word2vec で類似の単語をカタカナ表記してレーベンシュタイン距離を求めるデモです。単語を入力してください: ')
mecabtagger = MeCab.Tagger('-Oyomi -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
kata_target = mecabtagger.parse(target).strip()
word_list = [ent[0] for ent in w2v.most_similar(target,topn=topn)]
kata_word_list = [mecabtagger.parse(word).strip() for word in word_list]
kata_distances = [Levenshtein.distance(kata_target, kata_word) for kata_word in kata_word_list]
sorted_word_list = [word_list[i] for i in  np.argsort(kata_distances)] # [::-1]]

print(f'ターゲット語: {target} {kata_target}')
for i, word in enumerate(sorted_word_list):
    yomi = kata_word_list[word_list.index(sorted_word_list[i])]
    dist = kata_distances[word_list.index(sorted_word_list[i])] 
    print(f'{i+1:2d}: {word} (読み:{yomi}) 距離:{dist}')

word2vec で類似の単語をカタカナ表記してレーベンシュタイン距離を求めるデモです。単語を入力してください: 鬼滅の刃
ターゲット語: 鬼滅の刃 キメツノヤイバ
 1: 紅蓮の弓矢 (読み:グレンノユミヤ) 距離:6
 2: 竈門炭治郎 (読み:カマドタンジロウ) 距離:8
 3: 吾峠呼世晴 (読み:ゴトウゲコヨハル) 距離:8
 4: 呪術廻戦 (読み:ジュジュツカイセン) 距離:8
 5: 進撃の巨人 (読み:シンゲキノキョジン) 距離:8
 6: 約束のネバーランド (読み:ヤクソクノネバーランド) 距離:10
 7: ゆらぎ荘の幽奈さん (読み:ユラギソウノユウナサン) 距離:10
 8: 炎炎ノ消防隊 (読み:エンエンノショウボウタイ) 距離:11
 9: 僕のヒーローアカデミア (読み:ボクノヒーローアカデミア) 距離:12
10: 転生したらスライムだった件 (読み:テンセイシタラスライムダッタケン) 距離:15
